<a target="_blank" href="https://colab.research.google.com/github/Blaizzy/LLMOps/blob/main/inference/providers/anthropic/document_extraction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Getting started with Claude 3 family of LLMs

<img src="./assets/claude 3 thumbnail.JPG" width=500>

In this guide, you'll learn how to use Anthropic's Claude 3 models to extract information from large documents, including the vision variants.

Claude 3 offers a diverse range of model variants, each tailored to meet your specific requirements, ensuring optimal performance and speed:
- **Haiku** (Fastest) ⚡️: A lightning-fast model, ideal for time-sensitive tasks that demand rapid processing.
- **Sonnet** (Performance and Speed) 🚀: Striking the perfect balance between performance and speed, this variant excels in scenarios where both attributes are equally crucial.
- **Opus** (Best Performance) 🧠: The pinnacle of performance, this model variant is designed to tackle the most demanding and complex tasks.

Additionally, Claude 3 introduces groundbreaking multimodal variants, allowing you to seamlessly integrate image and text data for better information extraction and analysis.

### Essential Tools

To begin your journey with Claude 3, you'll need:

- **Anthropic LLM API**: Gain access to Anthropic's cutting-edge language models through their powerful API.
- **Langchain**: Leverage this versatile library to build robust and scalable applications powered by Claude 3's capabilities.


In [ ]:
!pip install -q langchain langchain-anthropic pypdf langchain_community anthropic python-dotenv

In [ ]:
from getpass import getpass
from dotenv import load_dotenv
from pprint import pprint
import os

# os.environ['ANTHROPIC_API_KEY'] = getpass() # create env variable directly
load_dotenv()

## Claude 3
<img src="./assets/claude 3 eval.webp" width=400>

- Displays increased capabilities in analysis, content creation, code generation and low resource language understanding.
- ~15% fewer refusals compared to previous models.
- ~20% improvement in accuracy on challenging open-ended questions.
- 200K context window with near-perfect recall, with up to 1M for selected users.
- Blazing-fast processing speed with Haiku variant capable of reading 10K token papers with images in under 3 seconds.

Read more <a href="./assets/Model_Card_Claude_3.pdf">here</a>.

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that responds like Snoop dogg."),
        ("human", "{message}")
    ]
)
chat = ChatAnthropic(temperature=0, model_name="claude-3-sonnet-20240229")
chain = prompt | chat

In [ ]:
output = chain.invoke({"message":"Hi"})
pprint(output.content)

Load PDF file

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

def load_file(file_name, file_type, n_pages=None):
    loader = PyPDFLoader(f"./assets/{file_name}.{file_type}")
    pages = loader.load_and_split()
    if n_pages:
        return pages[:n_pages]
    else:
        return pages

In [ ]:
document = load_file("Model_Card_Claude_3","pdf", 10)

In [ ]:
document

In [ ]:
pages_prompt = [f"<page number={i}>{page.page_content}</page>" for i, page in enumerate(document)]

In [ ]:
pages_prompt

In [ ]:
human = """
Please carefully review the given research paper and extract the following key details, enclosing them in the specified XML tags:

- The title of the research paper should be enclosed in <title></title> tags.
- The abstract or summary of the paper should be enclosed in <abstract></abstract> tags.
- Any mentioned models, algorithms, or techniques should be enclosed in individual <model></model> tags, with their respective capabilities or applications nested within <capabilities><capability></capability></capabilities> tags.
- Any benchmark results for the models should be enclosed in <benchmarks></benchmarks> tags, nested within the corresponding <model></model> tags. Each benchmark should be enclosed in <benchmark></benchmark> tags, with the associated score enclosed in <score></score> tags.

Example:
<title>Research Paper Title</title>
<abstract>This paper presents a novel approach to [...] The proposed method achieves [...] and outperforms existing techniques in terms of [...]</abstract>
<model>
    <name>Model A</name>
    <capabilities>
        <capability>Object detection</capability>
        <capability>Image classification</capability>
    </capabilities>
    <benchmarks>
        <benchmark>COCO Object Detection</benchmark>
        <score>0.78 mAP</score>
        <benchmark>ImageNet Classification</benchmark>
        <score>92.5% Top-5 Accuracy</score>
    </benchmarks>
</model>
<model>
    <name>Model B</name>
    <capabilities>
        <capability>Efficient text summarization</capability>
    </capabilities>
    <benchmarks>
        <benchmark>CNN/DailyMail Summarization</benchmark>
        <score>41.2 ROUGE-L</score>
    </benchmarks>
</model>

Please ensure that the extracted information accurately represents the key details from the research paper. If any of the requested details are not explicitly mentioned or available in the paper, leave the corresponding XML tags empty.

<document>
{pages}
</document>
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that returns XML."),
        ("human", human)
    ]
)

document_extraction_chain = prompt | chat

In [ ]:
output = document_extraction_chain.invoke({"pages": pages_prompt})
pprint(output.content)

## Claude 3 Vision

<img src="./assets/claude 3 vision eval.webp" width=400>

Load image in base64

Read more here:
- https://docs.anthropic.com/claude/docs/vision
- https://python.langchain.com/docs/integrations/chat/anthropic

### High-quality image

In [ ]:
import base64
from pathlib import Path

from IPython.display import HTML

img_path = Path("./assets/high_quality_img.png")
img_base64 = base64.b64encode(img_path.read_bytes()).decode("utf-8")

# Display image
HTML(f'<img src="data:image/png;base64,{img_base64}">')

In [ ]:
from langchain_core.messages import HumanMessage

def get_image_extraction_chain(img_base64):
    human = """
    Please carefully review the given image and extract the following key details, enclosing them in the specified XML tags:
    - Any mentioned models, algorithms, or techniques should be enclosed in individual <model></model> tags.
    - Any benchmark results for the models should be enclosed in <benchmarks></benchmarks> tags, nested within the corresponding <model></model> tags. Each benchmark should be enclosed in <benchmark></benchmark> tags, with the associated score enclosed in <score></score> tags.

    Example:
    <model>
        <name>Model A</name>
        <benchmarks>
            <benchmark>COCO Object Detection</benchmark>
            <score>0.78 mAP</score>
            <benchmark>ImageNet Classification</benchmark>
            <score>92.5% Top-5 Accuracy</score>
        </benchmarks>
    </model>
    <model>
        <name>Model B</name>
        <benchmarks>
            <benchmark>CNN/DailyMail Summarization</benchmark>
            <score>41.2 ROUGE-L</score>
        </benchmarks>
    </model>

    Please ensure that the extracted information accurately represents the key details from the research paper. If any of the requested details are not explicitly mentioned or available in the paper, leave the corresponding XML tags empty.
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant that returns XML."),
            HumanMessage(
                content=[
                    {
                        "type":"image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{img_base64}"
                        }
                    },
                    {"type": "text", "text": "{input}"}
                ]
            )
        ]
    )

    image_extraction_chain = prompt | chat
    return image_extraction_chain

In [ ]:
image_extraction_chain = get_image_extraction_chain(img_base64)
output = image_extraction_chain.invoke({"input": human})
pprint(output.content)

### Low-quality image

In [ ]:

img_path = Path("./assets/low_quality_img.png")
img_base64 = base64.b64encode(img_path.read_bytes()).decode("utf-8")

# Display image
HTML(f'<img src="data:image/png;base64,{img_base64}">')

In [ ]:
image_extraction_chain = get_image_extraction_chain(img_base64)
output = image_extraction_chain.invoke({"input": human})
pprint(output.content)

Function Calling and Tools:
- https://docs.anthropic.com/claude/docs/functions-external-tools
- https://python.langchain.com/docs/integrations/chat/anthropic_functions